In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
plt.rc('font', family='NanumBarunGothic')
matplotlib.font_manager._rebuild()

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체

In [3]:
import os

# 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomHorizontalFlip(), # augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = './data_10'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'Training'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'Test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=4, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=4, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

학습 데이터셋 크기: 1140
테스트 데이터셋 크기: 236
클래스: ['30017_롯데밀키스250ML', '30033_코카콜라250ML', '40007_롯데마운틴듀250ML', '40036_오란씨깔라만시250ML', '40049_코카환타오렌지250ML', '50012_일화맥콜250ML', '60008_일화천연사이다250ML', '60009_롯데펩시콜라250ML', '60037_코카환타파인애플250ML', '90118_코카콜라)스프라이트250ML']


In [4]:
model = models.resnet34(pretrained=True)
num_features = model.fc.in_features
# 전이 학습(transfer learning): 모델의 출력 뉴런 수를 10개로 교체하여 마지막 레이어 다시 학습
model.fc = nn.Linear(num_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
num_epochs = 20
model.train()
start_time = time.time()

# 전체 반복(epoch) 수 만큼 반복하며
for epoch in range(num_epochs):
    running_loss = 0.
    running_corrects = 0

    # 배치 단위로 학습 데이터 불러오기
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 모델에 입력(forward)하고 결과 계산
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.

    # 학습 과정 중에 결과 출력
    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

D:\things\Anaconda\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


#0 Loss: 1.5837 Acc: 46.4912% Time: 547.2811s
#1 Loss: 0.8510 Acc: 69.3860% Time: 1089.5099s
#2 Loss: 0.4617 Acc: 84.6491% Time: 1633.1342s
#3 Loss: 0.3034 Acc: 90.0877% Time: 2174.8002s
#4 Loss: 0.2410 Acc: 91.6667% Time: 2720.9685s
#5 Loss: 0.1836 Acc: 94.2982% Time: 3264.9137s
#6 Loss: 0.2035 Acc: 93.6842% Time: 3821.7342s
#7 Loss: 0.1104 Acc: 96.7544% Time: 4366.0907s
#8 Loss: 0.1353 Acc: 96.8421% Time: 4905.7046s
#9 Loss: 0.1100 Acc: 96.8421% Time: 5422.7297s
#10 Loss: 0.1163 Acc: 96.7544% Time: 5943.7597s
#11 Loss: 0.0441 Acc: 98.4211% Time: 6466.0056s
#12 Loss: 0.0940 Acc: 97.3684% Time: 6986.0368s
#13 Loss: 0.0824 Acc: 97.4561% Time: 7508.3513s
#14 Loss: 0.1445 Acc: 96.1404% Time: 8035.3117s
#15 Loss: 0.0652 Acc: 98.3333% Time: 8571.8592s
#16 Loss: 0.1113 Acc: 96.4912% Time: 9106.9372s
#17 Loss: 0.0469 Acc: 98.6842% Time: 9641.6011s
#18 Loss: 0.0230 Acc: 99.5614% Time: 10179.3594s
#19 Loss: 0.0602 Acc: 98.1579% Time: 10716.3439s


In [6]:
model.eval()
start_time = time.time()

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
        # print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
        # imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

    epoch_loss = running_loss / len(test_datasets)
    epoch_acc = running_corrects / len(test_datasets) * 100.
    print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))

[Test Phase] Loss: 4.4714 Acc: 52.9661% Time: 42.6342s


In [11]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [12]:
import torch
import torchvision
import torchsummary
torchsummary.summary(model, (3, 224, 224),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

training 할 때 validation 과정을 거치지 않아 모델 비교가 불가 -> validation 과정 넣어보자  
이 후 learning curve 그려볼 것 - https://androidkt.com/calculate-total-loss-and-accuracy-at-every-epoch-and-plot-using-matplotlib-in-pytorch/